In [1]:
import os

import keras

os.getcwd()
if '.git' not in os.listdir():
    os.chdir('./../../../../')
os.listdir()

import numpy as np
import pandas as pd
from transformers import AutoTokenizer, TFCamembertModel
import tensorflow as tf
import keras_core as keras

import tensorflow_hub as hub


2024-07-02 16:39:54.986004: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-02 16:39:54.986071: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-02 16:39:54.988266: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-02 16:39:55.191115: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Using TensorFlow backend


In [2]:
params_grid = {
    "embedder_data": "almanach-camembert-base",
    "dataset": "fr_hf.csv"
}


In [3]:
data_folder_src = 'ml/ml_core/data/processed/'
data_path_src = f'{data_folder_src}{params_grid["dataset"]}'

vector_path = f'ml/ml_core/embedded_vector/{params_grid["embedder_data"]}_{params_grid["dataset"]}'
vector_x_dst = f'{vector_path}.x.npy'
vector_y_dst = f'{vector_path}.y.npy'


## import CamemBERT model

In [4]:
tokenizer = AutoTokenizer.from_pretrained("almanach/camembert-base")

In [5]:
model = TFCamembertModel.from_pretrained("almanach/camembert-base")


2024-07-02 16:40:05.519571: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-02 16:40:05.756656: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-02 16:40:05.756700: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-02 16:40:05.758650: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-02 16:40:05.758712: I tensorflow/compile

In [6]:
def encode_texts(texts):
    inputs = tokenizer(texts, return_tensors="tf")
    outputs = model(inputs)
    outputs2 = outputs.last_hidden_state[:, 0, :]
    
    return outputs2

In [7]:

ds = pd.read_csv(data_path_src)


In [8]:

from tqdm import tqdm

tqdm.pandas(desc='Loading data')
ds['vector'] = ds['text'].progress_apply(encode_texts)

X = np.vstack(ds['vector'])
y = ds['labels'].values


Loading data: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 18071/18071 [36:58<00:00,  8.15it/s]


### saving embedding 

In [9]:
np.save(vector_x_dst, X)
np.save(vector_y_dst, y)